In [2]:
"""import os
localfilepath = "C:\\Users\\budde\\Desktop\\ds4300\\"
filepath = "practical_02_lon_kalden_desiree\\data\\"
word_docs = {}
file_list = os.listdir(localfilepath + filepath)
for file in file_list:
    words = []
    with open(localfilepath + filepath + file, mode="r") as infile:
        for line in infile.readlines():
            words = words + line.strip().split()
        word_docs[file] = words
word_docs"""

In [8]:
import os
import redis
from qdrant_client import QdrantClient
import chromadb
from sentence_transformers import SentenceTransformer
import ollama
redis_client = redis.Redis(host="localhost", port=6379, db=0)

In [10]:
# chunking function
def read_data(chunk_sizes=[200, 500, 1000], overlaps=[0, 50, 100]):
    filepath = "data/"
    # if we already processed it doesnt do it again
    if redis_client.keys("doc:*"):
        #print("Already preprocessed")
        return
    word_docs = {}

    file_list = os.listdir(filepath)
    
    for file in file_list:
        with open(filepath + file, mode="r", encoding='utf-8') as infile:
            file_str = infile.read().replace("\n", " ")  # remove newlines
        chunking = {}

        for chunk_size in chunk_sizes:
            for overlap in overlaps:
                chunks = []
                prev_idx = 0
                count = 1
                while prev_idx < len(file_str):
                    if prev_idx + chunk_size >= len(file_str):
                        chunk = file_str[prev_idx:]
                        word_docs[file + f' chunk{count}'] = chunk
                        break
                    else:
                        chunk = file_str[prev_idx:prev_idx + chunk_size]
                    chunks.append(chunk)
                    prev_idx = (prev_idx + chunk_size) - overlap
                
                chunking[f"chunk{chunk_size}_overlap{overlap}"] = chunks

        word_docs[file] = chunking
    return word_docs

In [11]:
# run chunks
word_docs = read_data()
#print(f"{len(word_docs)} docs processed")

✅ Preprocessed data already exists in Redis. Skipping processing.
